In [3]:
from confluent_kafka import DeserializingConsumer, KafkaError
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import StringDeserializer
from datetime import datetime
import uuid
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime
import pandas as pd

In [4]:
pip install confluent_kafka

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Create Spark session
spark = SparkSession.builder \
    .appName("Spark Project") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/16 05:21:17 INFO SparkEnv: Registering MapOutputTracker
23/10/16 05:21:17 INFO SparkEnv: Registering BlockManagerMaster
23/10/16 05:21:17 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/10/16 05:21:17 INFO SparkEnv: Registering OutputCommitCoordinator


In [6]:
# Define Kafka configuration
kafka_config = {
    'bootstrap.servers': 'YOUR SERVER',
    'sasl.mechanisms': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': 'YOUR USERNAME(API KEYS)',
    'sasl.password': 'YOUR PASSWORD(API KEYS PASSWORD)',
    'group.id':'group20',
    'auto.offset.reset': 'earliest'
}

In [7]:
# Create a Schema Registry client
schema_registry_client = SchemaRegistryClient({
  'url': 'ENDPOINT URL',
  'basic.auth.user.info': '{}:{}'.format('YOUR SCHEMA REGISTRY KEY', 'YOUR SCHEMA REGISTRY PASSWORD')
})

In [8]:
# Fetch the latest Avro schema for the value
subject_name = 'E-commerce_orders-value'
schema_str = schema_registry_client.get_latest_version(subject_name).schema.schema_str
print(schema_str)

{"type":"record","name":"Ecommerce","namespace":"com.kaggle.onlineretail","fields":[{"name":"order_id","type":"string"},{"name":"customer_id","type":"string"},{"name":"order_status","type":"string"},{"name":"order_purchase_timestamp","type":["long","int","string"]},{"name":"order_approved_at","type":["null","long","string"],"default":null},{"name":"order_delivered_carrier_date","type":["null","long","string"],"default":null},{"name":"order_delivered_customer_date","type":["null","long","string"],"default":null},{"name":"order_estimated_delivery_date","type":["null","long","string"],"default":null}]}


In [9]:
# Create Avro Deserializer for the value
key_deserializer = StringDeserializer('utf_8')
avro_deserializer = AvroDeserializer(schema_registry_client, schema_str)

# Define the DeserializingConsumer
consumer = DeserializingConsumer({
    'bootstrap.servers': kafka_config['bootstrap.servers'],
    'security.protocol': kafka_config['security.protocol'],
    'sasl.mechanisms': kafka_config['sasl.mechanisms'],
    'sasl.username': kafka_config['sasl.username'],
    'sasl.password': kafka_config['sasl.password'],
    'key.deserializer': key_deserializer,
    'value.deserializer': avro_deserializer,
    'group.id': kafka_config['group.id'],
    'auto.offset.reset': kafka_config['auto.offset.reset'],
    # 'enable.auto.commit': True,
    # 'auto.commit.interval.ms': 5000 # Commit every 5000 ms, i.e., every 5 seconds
})

In [10]:
consumer.subscribe(["E-commerce_orders"])

In [11]:
spark.sql("""set hive.exec.dynamic.partition.mode=nonstrict""")

23/10/16 05:21:40 WARN SetCommand: 'SET hive.exec.dynamic.partition.mode=nonstrict' might not work, since Spark doesn't support changing the Hive config dynamically. Please pass the Hive-specific config by adding the prefix spark.hadoop (e.g. spark.hadoop.hive.exec.dynamic.partition.mode) when starting a Spark application. For details, see the link: https://spark.apache.org/docs/latest/configuration.html#dynamically-loading-spark-properties.


DataFrame[key: string, value: string]

In [12]:
spark.sql("""USE class_db""")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


DataFrame[]

In [13]:
spark.sql("""CREATE TABLE IF NOT EXISTS ecommerce_orders (
    order_id STRING,
    customer_id STRING,
    order_status STRING,
    order_purchase_timestamp timestamp,
    order_approved_at timestamp,
    order_delivered_carrier_date timestamp,
    order_delivered_customer_date timestamp,
    order_estimated_delivery_date timestamp,
    order_hour int,
    order_day_of_week String
)""")

23/10/16 05:21:42 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


DataFrame[]

In [14]:
schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_status", StringType(), True),
    StructField("order_purchase_timestamp", TimestampType(), True),
    StructField("order_approved_at", TimestampType(), True),
    StructField("order_delivered_carrier_date", TimestampType(), True),
    StructField("order_delivered_customer_date", TimestampType(), True),
    StructField("order_estimated_delivery_date", TimestampType(), True),
    StructField("order_hour", IntegerType(), True),
    StructField("order_day_of_week", StringType(), True)    
])

In [15]:
def process_msg(msg):
    data=[]
    key = msg.key()
    value = msg.value()
    
    data.append(value["order_id"])
    data.append(value["customer_id"])
    data.append(value["order_status"])
    
    # Convert 'order_purchase_timestamp' string to datetime object
    order_purchase_timestamp_str = value['order_purchase_timestamp']
    order_purchase_timestamp = None
    if order_purchase_timestamp_str is not None:
        order_purchase_timestamp_str = order_purchase_timestamp_str +":00"
        order_purchase_timestamp = datetime.strptime(order_purchase_timestamp_str, '%d-%m-%Y %H:%M:%S')
        data.append(order_purchase_timestamp)
    else:
        data.append(order_purchase_timestamp_str)
    
    # Convert 'order_approved_at' to a datetime object
    order_approved_at_str = value['order_approved_at']
    order_approved_at = None
    if order_approved_at_str is not None:
        order_approved_at_str = order_approved_at_str + ":00"
        order_approved_at = datetime.strptime(order_approved_at_str, "%d-%m-%Y %H:%M:%S")
        data.append(order_approved_at)
    else:
        data.append(order_approved_at_str)
    
    # Convert 'order_delivered_carrier_date' to a datetime object
    order_delivered_carrier_date_str = value['order_delivered_carrier_date']
    order_delivered_carrier_date = None
    if order_delivered_carrier_date_str is not None:
        order_delivered_carrier_date_str = order_delivered_carrier_date_str + ":00"
        order_delivered_carrier_date = datetime.strptime(order_delivered_carrier_date_str, "%d-%m-%Y %H:%M:%S")
        data.append(order_delivered_carrier_date)
    else:
        data.append(order_delivered_carrier_date_str)
    
    # Convert 'order_delivered_customer_date' to a datetime object
    order_delivered_customer_date_str = value['order_delivered_customer_date']
    order_delivered_customer_date = None
    if order_delivered_customer_date_str is not None:
        order_delivered_customer_date_str = order_delivered_customer_date_str + ":00"
        order_delivered_customer_date = datetime.strptime(order_delivered_customer_date_str, "%d-%m-%Y %H:%M:%S")
        data.append(order_delivered_customer_date)
    else:
        data.append(order_delivered_customer_date_str)
    
    # Convert 'order_estimated_delivery_date' to a datetime object
    order_estimated_delivery_date_str = value['order_estimated_delivery_date']
    order_estimated_delivery_date = None
    if order_estimated_delivery_date_str is not None:
        order_estimated_delivery_date_str = order_estimated_delivery_date_str + ":00"
        order_estimated_delivery_date = datetime.strptime(order_estimated_delivery_date_str, "%d-%m-%Y %H:%M:%S")
        data.append(order_estimated_delivery_date)
    else:
        data.append(order_estimated_delivery_date_str)
    
    purchase_hour = order_purchase_timestamp.hour
    purchase_day_of_week = order_purchase_timestamp.strftime('%A')
    data.append(purchase_hour)
    data.append(purchase_day_of_week)
    data = [data]
    
    df = spark.createDataFrame(data, schema)
    try:
        df.select("order_id",
                  "customer_id",
                  "order_status",
                  "order_purchase_timestamp",
                  "order_approved_at",
                  "order_delivered_carrier_date" ,
                  "order_delivered_customer_date" ,
                  "order_estimated_delivery_date" ,
                  "order_hour",
                  "order_day_of_week").write.mode("append").insertInto("ecommerce_orders")
        
    except:
        print("An error occured")

    
    
    
    

In [ ]:
try:
    c=0
    while True:
        msg = consumer.poll(1.0)
        
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print("Reached end of partition")
            else:
                print("Error while consuming: {}".format(msg.error()))
                
        else:
            c=c+1
            process_msg(msg)
            print("Successfully consumed record with key {} and value {}".format(msg.key(), msg.value()))
            print(c)
            
            
except KeyboardInterrupt:
    pass

finally:
    consumer.close()

23/10/16 05:21:56 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


Successfully consumed record with key 31ad1d1b63eb9962463f764d4e6e0c9d_e69bfb5eb88e0ed6a785585b27e16dbf and value {'order_id': 'e69bfb5eb88e0ed6a785585b27e16dbf', 'customer_id': '31ad1d1b63eb9962463f764d4e6e0c9d', 'order_status': 'delivered', 'order_purchase_timestamp': '29-07-2017 11:55', 'order_approved_at': '29-07-2017 12:05', 'order_delivered_carrier_date': '10-08-2017 19:45', 'order_delivered_customer_date': '16-08-2017 17:14', 'order_estimated_delivery_date': '23-08-2017 00:00'}
1
Successfully consumed record with key 494dded5b201313c64ed7f100595b95c_e6ce16cb79ec1d90b1da9085a6118aeb and value {'order_id': 'e6ce16cb79ec1d90b1da9085a6118aeb', 'customer_id': '494dded5b201313c64ed7f100595b95c', 'order_status': 'delivered', 'order_purchase_timestamp': '16-05-2017 19:41', 'order_approved_at': '16-05-2017 19:50', 'order_delivered_carrier_date': '18-05-2017 11:40', 'order_delivered_customer_date': '29-05-2017 11:18', 'order_estimated_delivery_date': '07-06-2017 00:00'}
2
Successfully con

Successfully consumed record with key 72ae281627a6102d9b3718528b420f8a_2807d0e504d6d4894d41672727bc139f and value {'order_id': '2807d0e504d6d4894d41672727bc139f', 'customer_id': '72ae281627a6102d9b3718528b420f8a', 'order_status': 'delivered', 'order_purchase_timestamp': '03-02-2018 20:37', 'order_approved_at': '03-02-2018 20:50', 'order_delivered_carrier_date': '05-02-2018 22:37', 'order_delivered_customer_date': '08-02-2018 16:13', 'order_estimated_delivery_date': '21-02-2018 00:00'}
5
Successfully consumed record with key 62b423aab58096ca514ba6aa06be2f98_f3e7c359154d965827355f39d6b1fdac and value {'order_id': 'f3e7c359154d965827355f39d6b1fdac', 'customer_id': '62b423aab58096ca514ba6aa06be2f98', 'order_status': 'delivered', 'order_purchase_timestamp': '09-08-2018 11:44', 'order_approved_at': '10-08-2018 03:24', 'order_delivered_carrier_date': '10-08-2018 12:29', 'order_delivered_customer_date': '13-08-2018 18:24', 'order_estimated_delivery_date': '17-08-2018 00:00'}
6
Successfully con